In [41]:
import sys
import requests
import os
from pathlib import Path
from random import randint, random
from database import Persistence, CursorFromConnectionFromPool
from datetime import datetime
import pandas as pd
from psycopg2 import sql
import json
import utils

base_link = 'https://cpac.ca/'

def api_string(key, page, prg_id):
    return f'https://cpac.ca/api/1/services/contentModel.json?url=%2Fsite%2Fwebsite%2Fsearch%2Findex.xml&crafterSite=cpacca&key={key}&page={page}&type=all&order=desc&programId={prg_id}&tagId=undefined'

In [42]:
def get_api_data(key, page, prg_id):
    api = api_string(key, page, prg_id)

    search_results = requests.get(api).json()['page']['searchResult']
    data = search_results['item']
    total_pages = search_results['totalPages']
    data_lst = []
    for item in data:
        try:
            date_object = int(item['liveDateTime'].split('-')[0])
            if date_object >= 2020:
                data_lst.append({
                    'title': item['title_en_t'],
                    'description': item['description_en_t'],
                    'vid_link': base_link + item['url_en_s'],
                    'topics': {},
                    'top_topics': {},
                    'main_topic': '',
                    'with_title': {}
                })
        except Exception as e:
            pass
    if page < int(total_pages):
        return data_lst + get_api_data(key, page + 1, prg_id)
    else:
        return data_lst

con_data = get_api_data('conservative+mp', 1, 6) 
con_data_2 = get_api_data('conservative+leader+erin', 1, 6)
con_data_3 = get_api_data('conservative+leader+erin', 1, 92)

lib_data = get_api_data('liberal+mp', 1, 6) 
lib_data_2 = get_api_data('pm+justin', 1, 6)
lib_data_3 = get_api_data('pm+justin', 1, 92)

ndp_data = get_api_data('ndp+mp', 1, 6) 
ndp_data_2 = get_api_data('ndp+leader+jagmeet', 1, 6)
ndp_data_3 = get_api_data('ndp+leader+jagmeet', 1, 92)


In [43]:
data = ndp_data + lib_data + con_data + ndp_data_2 + lib_data_2 + con_data_2 + ndp_data_3 + lib_data_3 + con_data_3

In [44]:
print(len(ndp_data), len(lib_data), len(con_data))
print(len(ndp_data_2), len(lib_data_2), len(con_data_2))
print(len(ndp_data_3), len(lib_data_3), len(con_data_3))

39 17 31
84 10 52
67 20 18


In [46]:
# table = 'party_leader_video_summaries'
# data = ndp_data + lib_data + con_data
table = 'party_leader_video_summaries'

with CursorFromConnectionFromPool() as cur:
    try:
        create_table_query = sql.SQL(""" 
            CREATE TABLE IF NOT EXISTS {table} (
                title text UNIQUE,
                vid_link text,
                description text,
                topics json, 
                top_topics json,
                main_topic text,
                with_title json
            );

            ALTER TABLE {table} OWNER TO rds_ad;
        """).format(table=sql.Identifier(table))
        cur.execute(create_table_query)
        cur.connection.commit()
    except Exception as e:
        print(f'An exception occured executting a query:\n{e}')
        cur.connection.rollback()

    insert_legislator_query = sql.SQL("""
        INSERT INTO {table}
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (title) DO UPDATE SET
            vid_link = excluded.vid_link,
            description = excluded.description,
            topics = excluded.topics,
            top_topics = excluded.top_topics,
            main_topic = excluded.main_topic,
            with_title = excluded.with_title
        """).format(table=sql.Identifier(table))

    for item in data:
        try:
            tup = (
                item['title'],
                item['vid_link'],
                item['description'],
                json.dumps(item['topics'], default=utils.json_serial),
                json.dumps(item['top_topics'], default=utils.json_serial),
                item['main_topic'],
                json.dumps(item['with_title'], default=utils.json_serial)
            )

            cur.execute(insert_legislator_query, tup)
        except Exception as e:
            print(f'Exception occured inserting the following data:\n{tup}')
            print(e)
            cur.connection.rollback()
    print('done!')

done!
